In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

file_path = "/kaggle/input/global-air-pollution-dataset/global air pollution dataset.csv"
data = pd.read_csv(file_path)

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = data.dropna()

# Множинний регресійний аналіз
Для прогнозування `AQI Value` побудуємо множинну регресійну модель на основі `CO AQI Value`, `Ozone AQI Value`, `NO2 AQI Value` і `PM2.5 AQI Value`.

In [2]:
X = data[['CO AQI Value', 'Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']]
y = data['AQI Value']

X = sm.add_constant(X)

# Fit the multiple regression model
model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              AQI Value   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                 2.228e+05
Date:                Fri, 05 Jul 2024   Prob (F-statistic):               0.00
Time:                        09:07:16   Log-Likelihood:                -83158.
No. Observations:               23035   AIC:                         1.663e+05
Df Residuals:                   23030   BIC:                         1.664e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.6432      0.112     -5.721      0.000      -0.864      -0.423
CO AQI Value        0.0109      0.040      0.273      0.785      -0.067       0.089
Ozone AQI Value     0.1556      0.002     66.208      0.000       0.151       0.160
NO2 AQI Value      -0.0368      0.014     -2.714      0.007      -0.063      -0.010
PM2.5 AQI Value     0.9814      0.001    772.464      0.000       0.979       0.984
==============================================================================
Omnibus:                    22523.913   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1847377.782
Skew:                           4.633   Prob(JB):                         0.00
Kurtosis:                      45.883   Cond. No.                         181.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Результат показує що:
- значення R-квадрат = 0,975: 97,5% дисперсії «AQI Value» можна пояснити моделлю.
- ключові параметри для прогнозування: `Ozone AQI Value`, `NO2 AQI Value` і `PM2.5 AQI Value` є статистично важливими параметрами для прнозу, зі значеннями p менше 0,05. `CO AQI Value` незначне.

# Аналіз еластичності
Розрахуємо еластичність для кожного ключового показника прогнозування

Еластичність = $\beta_i \times \frac{\bar{x_i}}{\bar{y}}$

 $\beta_i$ — коефіцієнт незалежної змінної.

 $\bar{x_i}$ — середнє значення незалежної змінної.

 $\bar{y}$ — середнє значення залежної змінної.
.

In [3]:
# Calculate means of independent variables and dependent variable
means = X.mean()
mean_y = y.mean()

# Calculate elasticity
elasticity = model.params * (means / mean_y)

# Filter for significant predictors
significant_predictors = ['Ozone AQI Value', 'NO2 AQI Value', 'PM2.5 AQI Value']
elasticity = elasticity[significant_predictors]

print(elasticity)

Ozone AQI Value    0.075802
NO2 AQI Value     -0.001569
PM2.5 AQI Value    0.934452
dtype: float64


Отримані результати:

- `Ozone AQI Value`: еластичність 0,075802 означає, що збільшення на 1% `Ozone AQI Value` пов’язане зі збільшенням `AQI Value` приблизно на 0,0758%. Це вказує на відносно низьку, але позитивну чутливість AQI до змін рівня озону.
- `NO2 AQI Value`: еластичність -0,001569 означає, що збільшення на 1% у `NO2 AQI Value` пов’язане зі зниженням `AQI Value` приблизно на 0,0016%. Це свідчить про дуже малу негативну залежність між рівнями NO2 і загальним AQI, тобто зі збільшенням NO2 загальний AQI трохи знижується. Однак цей ефект дуже мінімальний.
- `PM2.5 AQI Value`: еластичність 0,934452 означає, що збільшення на 1% `PM2.5 AQI Value` пов’язане зі збільшенням `AQI Value` приблизно на 0,9345%. Це вказує на дуже високу чутливість AQI до змін рівнів PM2,5, показуючи, що PM2,5 має значний вплив на загальний індекс якості повітря.

Судячи з результатів серед виміряних забруднювачів PM2.5 має найбільш значний вплив на індекс якості повітря, за ним йде озон, тоді як NO2 має незначний вплив.

# Перевірка мультиколінеарності

In [4]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Calculate VIF for each predictor
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

           feature       VIF
0            const  3.638606
1     CO AQI Value  1.551141
2  Ozone AQI Value  1.267993
3    NO2 AQI Value  1.475641
4  PM2.5 AQI Value  1.408115


Дивлячись на отримані результати усі значення нижчі за 5, що вказує напомірну мультиколінеарність, прогностичні змінні не сильно корельовані одна з одною.